In [5]:
import FinanceDataReader as fdr

name = "삼성전자"
df_krx = fdr.StockListing('KRX')
target = df_krx[df_krx['Name'] == name]
target

code = target.iloc[0]['Code']
price = target.iloc[0]['Close']
market_cap = target.iloc[0]['Marcap'] / 10**8  # 억 단위

print(f"종목명: {name}")
print(f"종목코드: {code}")
print(f"현재가: {price}원")
print(f"시가총액: {market_cap:.2f} 억원")


종목명: 삼성전자
종목코드: 005930
현재가: 152100원
시가총액: 9003769.28 억원


In [ ]:
import FinanceDataReader as fdr
import requests
from bs4 import BeautifulSoup

def get_financial_data(code):
    """네이버 증권에서 주요 재무 지표를 크롤링합니다."""
    url = f"https://finance.naver.com/item/main.naver?code={code}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    metrics = {}
    try:
        # 네이버 증권의 주요재무정보 테이블(표)을 찾습니다.
        table = soup.select_one(".section.cop_analysis div.sub_section table")
        # 항목들 추출 (ROE, 부채비율, PSR 계산을 위한 매출액 등)
        # 실제 운영시에는 행/열 인덱스를 정교하게 매칭해야 합니다.
        rows = table.select("tbody tr")
        
        # 현재주가, 시가총액, 증거금률, 52주 최저가, 52주 최고가,
        # PER, PBR, PSR, PCR, PEG, ROE, ROA
        # 영업이익률, 순이익률, 매출액 증가율, 순이익 증가율, 부채비율, 유동비율, 유보율, 배당수익률, 외국인 지분율

        data['sales'] = rows[0].select("td")[3].text.strip().replace(',', '') # 최근 매출액
        data['roe'] = rows[5].select("td")[3].text.strip().replace(',', '')
        data['debt_ratio'] = rows[6].select("td")[3].text.strip().replace(',', '')
        data['eps'] = rows[9].select("td")[3].text.strip().replace(',', '')
        data['per'] = rows[10].select("td")[3].text.strip().replace(',', '')
        data['bps'] = rows[11].select("td")[3].text.strip().replace(',', '')
        data['pbr'] = rows[12].select("td")[3].text.strip().replace(',', '')

# --- Market Info ---
        metrics['current_price'] = safe_convert(soup.select_one(".no_today .blind").text)
        metrics['market_cap'] = safe_convert(soup.select_one("#_market_sum").text.replace("조", "").replace("억원", ""))
        
        # 52-week High/Low
        week_52_data = soup.select(".tab_con1 .blind")
        metrics['week_52_low'] = safe_convert(week_52_data[0].text) if len(week_52_data) > 0 else None
        metrics['week_52_high'] = safe_convert(week_52_data[1].text) if len(week_52_data) > 1 else None
        
        # Foreign Ownership
        foreign_el = soup.select_one(".gray .lft th:contains('외국인소진율') + td")
        metrics['foreign_ratio'] = safe_convert(foreign_el.text) if foreign_el else None

        # --- Financial Analysis (cop_analysis table) ---
        table = soup.select_one(".section.cop_analysis div.sub_section table")
        rows = table.select("tbody tr")
        
        # Mapping by row index (Based on Naver Finance standard layout)
        # Column [3] usually represents the latest annual data
        metrics['sales'] = safe_convert(rows[0].select("td")[3].text)
        metrics['operating_profit_margin'] = safe_convert(rows[1].select("td")[3].text)
        metrics['net_profit_margin'] = safe_convert(rows[2].select("td")[3].text)
        metrics['roe'] = safe_convert(rows[5].select("td")[3].text)
        metrics['debt_ratio'] = safe_convert(rows[6].select("td")[3].text)
        metrics['current_ratio'] = safe_convert(rows[7].select("td")[3].text)
        metrics['reserve_ratio'] = safe_convert(rows[8].select("td")[3].text)
        metrics['per'] = safe_convert(rows[10].select("td")[3].text)
        metrics['pbr'] = safe_convert(rows[12].select("td")[3].text)
        metrics['dividend_yield'] = safe_convert(rows[13].select("td")[3].text)

        # --- Calculated Metrics ---
        # PSR = Market Cap / Sales
        if metrics['market_cap'] and metrics['sales']:
            metrics['psr'] = round(metrics['market_cap'] / metrics['sales'], 2)
        else:
            metrics['psr'] = None

        # ROA / PCR / PEG / Sales Growth / Profit Growth
        # Note: These may require cross-referencing previous year data for growth rates.
        # For now, setting placeholder or simplified logic.
        metrics['roa'] = None  # Requires 'Financials' tab for precision
        metrics['pcr'] = None
        metrics['peg'] = None
        metrics['sales_growth'] = None
        metrics['profit_growth'] = None
        metrics['margin_rate'] = None # Example: Margin requirement (brokerage specific)        
    except Exception as e:
        print(f"데이터 추출 중 오류: {e}")
    return data

In [7]:
fin_data = get_financial_data("005930")
print(fin_data)

{'roe': '9.53', 'debt_ratio': '', 'sales': '3291027'}
